In [ ]:
!pip install lightfm

In [ ]:
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

In [ ]:
# 📥 Étape 1 : Charger MovieLens
url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
columns = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(url, sep='\t', names=columns)

# Take only if he likes movie
df = df[df['rating'] >= 4]

# 🧼 Convertir les IDs en strings
# Convertir les IDs en string AVANT de les passer à LightFM
df['user_id'] = df['user_id'].astype(str)
df['item_id'] = df['item_id'].astype(str)


# Split en 80% train / 20% test
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)


<ipython-input-4-6d93c912389d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_id'] = df['user_id'].astype(str)
<ipython-input-4-6d93c912389d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['item_id'] = df['item_id'].astype(str)


In [ ]:
# Fit uniquement sur le TRAIN pour ne pas biaise le TEST set
dataset = Dataset()
dataset.fit(df['user_id'], df['item_id'])  # Fit sur tout pour garder tous les IDs connus

# Matrice d’entraînement
interactions_train, _ = dataset.build_interactions([
    (row.user_id, row.item_id) for row in df_train.itertuples()
])

# Matrice de test (à utiliser avec precision@k ou auc_score)
interactions_test, _ = dataset.build_interactions([
    (row.user_id, row.item_id) for row in df_test.itertuples()
])

In [ ]:
from lightfm.evaluation import precision_at_k

epochs = 30
train_precisions = []
test_precisions = []
"""
for epoch in range(epochs):
    model.fit_partial(interactions_train, epochs=1, num_threads=2)
    train_precision = precision_at_k(model, interactions_train, k=5).mean()
    test_precision = precision_at_k(model, interactions_test, k=5).mean()

    train_precisions.append(train_precision)
    test_precisions.append(test_precision)

    print(f"Epoch {epoch+1}: Train P@5 = {train_precision:.4f} | Test P@5 = {test_precision:.4f}")"""
model = LightFM(loss='warp')
model.fit(interactions_train, epochs=12, num_threads=2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from lightfm.evaluation import precision_at_k, auc_score

print("📈 Precision@5 :", precision_at_k(model, interactions_test, k=5).mean())
print("🎯 AUC Score :", auc_score(model, interactions_test).mean())

📈 Precision@5 : 0.104451686
🎯 AUC Score : 0.9033642


Déclaration de la fonction :

model : le modèle LightFM entraîné

dataset : l'objet Dataset() de LightFM (contenant les mappings utilisateurs/items)

user_id_str : ID utilisateur en string (ex. : '1')

top_n : nombre de recommandations à renvoyer (par défaut : 5)



In [ ]:
# ✅ Fonction de recommandation avec affichage des titres de films
def recommend(model, dataset, user_id_str, movie_df, top_n=5,normalize=True):
    # Récupère les mappings internes LightFM
    # user_id_map : {'1': 0, '2': 1, ...}
    # item_id_map : {'50': 0, '174': 1, ...}
    user_id_map, _, item_id_map, _ = dataset.mapping()

    # Vérifie si l'utilisateur est présent dans le dataset
    if user_id_str not in user_id_map:
        print("Utilisateur inconnu.")
        return []

    # Convertit l'ID utilisateur (ex. '1') en index numérique interne (ex. 0)
    user_idx = user_id_map[user_id_str]

    # Récupère la liste des IDs réels des films (ex. ['50', '174', '56', ...])
    item_ids = list(item_id_map.keys())

    # Récupère les indices internes (entiers) des films (ex. [0, 1, 2, ...])
    item_indices = list(item_id_map.values())

    # Prédit un score pour chaque film pour l'utilisateur donné
    # Renvoie un tableau de scores (plus c'est haut, plus c'est pertinent)
    scores = model.predict(user_idx, np.array(item_indices))
    #  Normalisation (optionnelle) des scores entre 0 et 1
    if normalize:
        scaler = MinMaxScaler()
        scores = scaler.fit_transform(scores.reshape(-1, 1)).flatten()
    # Trie les scores par ordre décroissant et garde les indices des top-N items
    top_indices = np.argsort(scores)[::-1][:top_n]

    # Récupère les IDs réels des items les mieux notés
    top_items = [item_ids[i] for i in top_indices]

    # Filtre le DataFrame movie_df pour ne garder que les films recommandés
    recommended_movies = movie_df[movie_df['item_id'].isin(top_items)].copy()

    # Crée une colonne "rank" pour garder l'ordre original de top_items
    recommended_movies['rank'] = recommended_movies['item_id'].apply(lambda x: top_items.index(x))

    # Trie les films selon leur rang dans le top N
    recommended_movies = recommended_movies.sort_values('rank')

    # Affiche les titres des films avec leur score
    for idx, row in recommended_movies.iterrows():
        # Récupère le score à partir de l'index dans item_ids
        score = scores[item_ids.index(row['item_id'])]

        # Affichage propre : 🎬 Titre (ID: 50) → Score: 0.9321
        print(f"🎬 {row['title']} (ID: {row['item_id']}) → Score: {round(score, 4)}")

    # Renvoie un DataFrame propre avec les films recommandés
    return recommended_movies

In [ ]:
# 📁 Charger les titres de films (u.item)
movie_url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.item"
movie_df = pd.read_csv(movie_url, sep='|', encoding='latin-1', header=None)
movie_df = movie_df[[0, 1]]
movie_df.columns = ['item_id', 'title']
movie_df['item_id'] = movie_df['item_id'].astype(str)  # important


In [ ]:
# 🧪 Exemple : recommandations pour l'utilisateur '2'
recommend(model, dataset, user_id_str='2', movie_df=movie_df, top_n=5)

🎬 English Patient, The (1996) (ID: 286) → Score: 1.0
🎬 Titanic (1997) (ID: 313) → Score: 0.953499972820282
🎬 Contact (1997) (ID: 258) → Score: 0.939300000667572
🎬 Full Monty, The (1997) (ID: 269) → Score: 0.9222000241279602
🎬 L.A. Confidential (1997) (ID: 302) → Score: 0.885200023651123


,item_id,title,rank
285,286,"English Patient, The (1996)",0
312,313,Titanic (1997),1
257,258,Contact (1997),2
268,269,"Full Monty, The (1997)",3
301,302,L.A. Confidential (1997),4
